In [1]:
from music.models import MusicGadgetIc
from core.models import Category
from music.utils import setup_run
from gadget.models import GadgetRun

import numpy as np

from hog import settings

from astropy import units as u


from tqdm import tqdm_notebook

In [2]:
# cat = Category.objects.filter(name='selection 2')
# cat = Category.objects.get(name='Lim_2Rvir')
# cat = Category.objects.get(name='Lim_4Rvir')
cat = Category.objects.get(name='MARE19')

ics = MusicGadgetIc.objects.filter(category=cat)

In [3]:
def get_ic_location(ic, music_media='/fast_scratch1/lbignone/hog/media/'):

    root = settings.MEDIA_ROOT

    path = ic.get_path()
    path_strip = path[len(root):]
    ic_filename = music_media + path_strip + str(ic.id) + '_' + ic.filename

    return ic_filename

In [4]:
def estimate_softening(boxlength, level, factor=1./25):
    d = boxlength / (2**level)
    return d/25

In [5]:
def get_maximum_phys_softening(softening, redshift_limit=2.91):
    a_lim = 1./(1. + redshift_limit)
    return a_lim*softening

In [6]:
media_root = '/fast_scratch1/lbignone/hog/media/'

In [7]:
def set_gr(gr=None, ic=None, music_media='/fast_scratch1/lbignone/hog/media/'):
    
    # if no gr is provided create one based on the provided ic
    if gr is None:
    
        gr = GadgetRun()
        gr.ic = ic
    
        gr.save()
    
        name = ic.name
        
        gr.name = name
        gr.category.add(cat)
        
    else:
        ic = gr.ic
    
    name_id = str(gr.id) + '_' + gr.name
    
    # makefile options
    gr.DOUBLEPRECISION = True
    gr.DOUBLEPRECISION_FFTW = True
    gr.UNEQUALSOFTENINGS = True
    gr.OUTPUTPOTENTIAL = True
    gr.PLACEHIGHRESREGION = 3
    gr.ENLARGEREGION = 1.2
    
    # Filenames and file formats 
    gr.OutputDir = media_root + '/gadget/run/' + name_id
    gr.file_number = 10    
    gr.InitCondFile = get_ic_location(ic, music_media=music_media)
    
    # Simulation specific parameters
    gr.TimeBegin = 1./(1.+ic.zstart)
    gr.boxlength = ic.boxlength * u.Mpc.to('kpc')
    gr.Omega_m = ic.Omega_m
    gr.Omega_l = ic.Omega_L
    gr.Omega_b = ic.Omega_b
    gr.h = ic.H0 / 100
    
    # Output of snapshot files 
    gr.OutputListOn = True
    gr.OutputListFilename = '/home/lbignone/wmmw/ExpansionList_64'
    gr.TimeBetStatistics = 0.05
    gr.NumFilesWrittenInParallel = 10
    
    levelmax = ic.levelmax
    levelmin = ic.levelmin
    
    softenings = []
    for level in range(levelmin, levelmax+1):
        softening = estimate_softening(gr.boxlength, level)
        softenings.append(softening)
    softenings = np.array(softenings)
           
    # High-resolution softenings
    gr.SofteningGas = softenings[-1]
    gr.SofteningHalo = softenings[-1]
    gr.SofteningStars = softenings[-1]
    
    # low-resolution softenings
    gr.SofteningBndry = softenings[0]
    
    # medium-resolution softenings
    gr.SofteningDisk = softenings[-2]
    
    medium_softening = softenings[1:-2].sum() / len(softenings[1:-2])
            
    gr.SofteningBulge = medium_softening
    
    gr.SofteningGasMaxPhys = get_maximum_phys_softening(gr.SofteningGas)
    gr.SofteningHaloMaxPhys = get_maximum_phys_softening(gr.SofteningHalo)
    gr.SofteningDiskMaxPhys = get_maximum_phys_softening(gr.SofteningDisk)
    gr.SofteningBulgeMaxPhys = get_maximum_phys_softening(gr.SofteningBulge)
    gr.SofteningStarsMaxPhys = get_maximum_phys_softening(gr.SofteningStars)
    gr.SofteningBndryMaxPhys = get_maximum_phys_softening(gr.SofteningBndry)
    
    gr.save()



In [30]:
ics = MusicGadgetIc.objects.filter(category=cat)
# ics = ics[13:14]

ics = ics.filter(name='229533_DM_L12')

ics = [ics[1]]

In [31]:
for ic in ics:
    set_gr(gr=None, ic=ic)

In [32]:
from gadget.utils import save_makefile, save_config, save_pbs_file, transfer_to_host

In [33]:
grs = GadgetRun.objects.filter(category=cat, name='229533_DM_L12')
grs

<PolymorphicQuerySet [<GadgetRun: 229533_DM_L12>, <GadgetRun: 229533_DM_L12>, <GadgetRun: 229533_DM_L12>]>

In [34]:
gr = grs[2]
gr

<GadgetRun: 229533_DM_L12>

In [35]:
save_makefile(gr, systype='Geryon2_gnu')
save_config(gr)
save_pbs_file(gr, nodes=3, ppn=40, template='geryon.pbs', module_load_list='gnu')
transfer_to_host(gr, host='geryon2', media_path='/fast_scratch1/lbignone/hog/media')

In [16]:
save_makefile(gr, systype='NLHPC_intel')
save_config(gr)

modules = 'module load intel impi fftw_intel/2.1.5 gsl'

save_pbs_file(gr, nodes=6, ppn=20, template='nlhpc.pbs', module_load_list=modules)
transfer_to_host(gr, host='leftraru.nlhpc.cl', media_path='/home/lbignone/hog/media/')


In [15]:
gr

<GadgetRun: 181619_DM_L12>